In [55]:
import numpy as np
import scipy as sp
import scipy.stats
import csv

In [2]:
[np.random.normal(50, 10) for n in range(20)]

[56.859269900339584,
 59.3545335933515,
 65.39598133590218,
 64.94660132373342,
 45.367148695372435,
 58.536792938732084,
 40.09072672428736,
 62.58909398520636,
 69.5278583810341,
 48.045197773199625,
 45.335305733456835,
 58.26293985205911,
 37.81026358321324,
 47.40490838270374,
 47.40297207603848,
 41.364368703765095,
 37.30792719798016,
 43.21912134357436,
 51.327346723438886,
 54.55403370671512]

In [3]:
def boundedRandom(mu, sigma, top, bottom):
    return sorted([top, np.random.normal(mu, sigma), bottom])[1]

In [172]:
class Legislator:
    def __init__(self, alignment, reliability=0.2, majority=False):
        self.mu = alignment
        self.sigma = reliability
        self.majority = majority
    def vote(self):
        return np.random.normal(self.mu, self.sigma)
#         return boundedRandom(self.mu, self.sigma, 0, 1)

In [5]:
l = Legislator(0.3)
[l.vote() for v in range(20)]

[0.32617997765040474,
 0.2702640139329612,
 0.07190758622369459,
 0.40749692359207834,
 0.13788048304746106,
 0.5849231359796203,
 0.6662848372908419,
 0.4770355116271705,
 0,
 0.06488181099196011,
 0.4500230484790619,
 0.4635269845276313,
 0.4683179280112731,
 0.5013340518453754,
 0.11406345475251492,
 1,
 0.7243475897288714,
 0.42721117827069477,
 0.36331298604445933,
 0.6017738337086318]

In [12]:
class Result:
    def __init__(self, chamber):
        self.threshold = len(chamber) / 2
        self.votes = [l.vote() for l in chamber]
        self.result = len([v for v in self.votes if v >= 0.5])
    def __repr__(self):
        return str(self.result)
    def carried(self):
        return self.result > self.threshold

In [174]:
class Legislature:
    def __init__(self, members, majority, polarization=0.2, uniformity=0.2, eccentricity=0.1):
        self.members = members
        self.majority = majority
        minority = members - majority
        self.minority = minority
        
        if(minority < 0):
            raise ValueError('Invalid legislature, majority larger than chamber')
            
        if(polarization > 0.5):
            raise ValueError('Polarization too large')
        if(polarization < 0):
            raise ValueError('Polarization too small')
        
        majorityCenter = 0.5 + polarization
        minorityCenter = 0.5 - polarization
        
        chamberMajority = [Legislator(
                np.random.normal(majorityCenter, uniformity),
#                 boundedRandom(majorityCenter, uniformity, 0, 1),
                eccentricity, True
            ) for n in range(majority)]
        chamberMinority = [Legislator(
                np.random.normal(majorityCenter, uniformity),
#                 boundedRandom(minorityCenter, uniformity, 0, 1),
                eccentricity, False
            ) for n in range(minority)]
        self.chamber = chamberMajority + chamberMinority
    def vote(self):
        return Result(self.chamber)

In [146]:
l = Legislature(9,5)
[(mp.mu, mp.sigma) for mp in l.chamber]

[(0.7481403502473992, 0.1),
 (0.44237187134755895, 0.1),
 (0.42020997059582765, 0.1),
 (1, 0.1),
 (0.8783596012268905, 0.1),
 (0.19302109108557713, 0.1),
 (0.2440781070287973, 0.1),
 (0.4727252939963481, 0.1),
 (0.229523419826486, 0.1)]

In [154]:
class test:
    def __init__(self, legislature, votes=20):
        self.votes = votes
        self.results = [legislature.vote() for n in range(votes)]
    def carried(self):
        return len([r for r in self.results if r.carried() is True]) / self.votes
    def __repr__(self):
        return str(self.carried())

In [155]:
t = test(l,20)
t.results

[3, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 3, 4, 3, 4, 4, 4, 5, 4, 5]

In [159]:
def runLegislature(n, votes=20, *args, **kwargs):
    legislatures = [Legislature(*args, **kwargs) for n in range(n)]
    results = [test(l, votes) for l in legislatures]
    if(n < 20):
        print(results)
    results_num = [float(r.carried()) for r in results]
    print(np.median(results_num))
    print(sp.stats.describe(results_num))
    return results

In [182]:
smallRun = runLegislature(4000, 20, 9, 5, 0.1)

0.95
DescribeResult(nobs=4000, minmax=(0.0, 1.0), mean=0.86483749999999981, variance=0.044704149631157784, skewness=-1.9149981490769823, kurtosis=3.100872218020471)


In [183]:
largeRun = runLegislature(4000, 20, 900, 500, 0.1)

1.0
DescribeResult(nobs=4000, minmax=(1.0, 1.0), mean=1.0, variance=0.0, skewness=0.0, kurtosis=-3.0)


In [177]:
with open('./smallRun.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows([r.results for r in smallRun])

In [178]:
with open('./largeRun.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerows([r.results for r in largeRun])